# Modelo Algortimo SVM

## Definicion

Es un algoritmo de clasificaion como de regresion utilizado en base a los hiperplanos . La idea es poder separar intuitivamente con un hiperplano los datos y asi llegar a la mejor conversion.


## Explicacion con ejemplo

si tuvieramos un conjunto de datos asi

![inicial](https://miro.medium.com/max/1280/1*VDATmWG1E1ZNg7hdasOh5g.png)

Vemos que existe muchos opciones para poder separarlos dado que entre todos los puntos rojos y azules hay como un monto de rectas(hiperplano para dos dimensiones) que podriamos crear.

El tema es poder la mejor recta que maximice los vectores ortogonales a la recta principal de separacion

![marginsvm](https://miro.medium.com/max/1280/1*irg_jfdAar9gfe0j-Q04vQ.png)


Como se ve en la foto se maximiza el margen con los vectores , haciendo mas representativo la seleccion, cualqueir otra recta hubiera sido un problema dado que posiblemente no represente bien la agrupacion de valores

##Parametros

Los parametros mas usados o necesarios en SVM son

* **C** que es el regularizador de la suavidad del margen , quiero decir que el margen calculado arriba acepta mas errores y hace que se mas ancho ese bondad para poder marcar mejor el hiperplano mejor. Entre mas chico el C mas docil

* **kernel** seleccion de la creacion del hiperplano , donde seleccionamos su funcion matematica a armar como margen. Ejemplo funcion lineal ,  funcion polinomica , rbf(Radial Basis Function)

* **gamma** es un numero scalar que que determina el relacionamiento entre puntos dando un relacionamiento contra el algoritmo

![gammayC](https://scikit-learn.org/stable/_images/sphx_glr_plot_rbf_parameters_001.png)


##Conclusiones

* Ventajas

Se puede observar que SVM es muy utilizado en clasificaiocn de texto , dado que es excelente para poder trabajar con alta dimensionalidad pero con menos samples.
Tambien cuando no hay un relacionamineto lineal entre input y output

* Desventajas

No sirve mucho cuando los datos son muy grandes , es decir el dataset tiene muchas samples





#ejemplo y codigo

In [1]:
import pandas as pd
import numpy as np

In [2]:
#load dataset
airlines = pd.read_csv('airlines_sample.csv')
airlines.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
363020,363021,DL,2582,MCO,BDL,2,1155,153,0
422163,422164,WN,2772,SAT,BNA,6,645,125,0
318247,318248,MQ,3044,LAX,DEN,7,725,140,0
260117,260118,WN,3496,LIT,PHX,4,420,195,0
294548,294549,DL,1924,ATL,BWI,5,1290,113,0


In [3]:
airlines.shape

(1079, 9)

In [4]:
#chequeo de nulos
airlines.isnull().sum()


id             0
Airline        0
Flight         0
AirportFrom    0
AirportTo      0
DayOfWeek      0
Time           0
Length         0
Delay          0
dtype: int64

Chequeo de variables determinadno cuales son categoricas y cuales numericas

In [ ]:
#chequeo de variables
airlines.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1079 entries, 363020 to 339169
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1079 non-null   int64 
 1   Airline      1079 non-null   object
 2   Flight       1079 non-null   int64 
 3   AirportFrom  1079 non-null   object
 4   AirportTo    1079 non-null   object
 5   DayOfWeek    1079 non-null   int64 
 6   Time         1079 non-null   int64 
 7   Length       1079 non-null   int64 
 8   Delay        1079 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 84.3+ KB


In [ ]:
airlines.describe().T

,count,mean,std,min,25%,50%,75%,max
id,1079.0,270299.117702,149871.383642,181.0,140333.5,270039.0,397616.0,539149.0
Flight,1079.0,2474.482854,2014.988020,2.0,772.5,1947.0,3770.5,7799.0
DayOfWeek,1079.0,3.828545,1.936679,1.0,2.0,4.0,5.0,7.0
Time,1079.0,812.069509,278.988209,30.0,565.0,810.0,1040.0,1439.0
Length,1079.0,128.705283,66.642459,27.0,80.0,110.0,155.5,410.0
Delay,1079.0,0.445783,0.497282,0.0,0.0,0.0,1.0,1.0


In [18]:
#seleccion de variables categoricas

cat_columns = airlines.select_dtypes(include='object').columns#guardo en una variable el nombre de las columnas object

In [10]:
numeric_columns = airlines.select_dtypes(include=np.number).columns#guardo en una variable el nombre de las columnas numero

Vamos a chequear la cantidad de valores diferentes en cada categorica,  dado que es importante para evaluar que metodo de trasnforqcion vy a utilizar

In [ ]:
[] # lista, mutable
() # tupla (airline , 10 , 'lax', 'mvd',....), inmutable

In [20]:
cat_columns

Index(['Airline', 'AirportFrom', 'AirportTo'], dtype='object')

In [21]:
#chequeo de dimensionalidad de variables categoricas
lista = []
#loop por columnas categoricas for unidad in loquequieroquerrecorra
for col in cat_columns:
  info = (col, airlines[col].nunique(), airlines[col].unique() )
  lista.append(info)
pd.DataFrame(lista, columns=['feature', 'cantidad', 'valores'])

,feature,cantidad,valores
0,Airline,18,"[DL, WN, MQ, 9E, US, EV, CO, OO, B6, FL, UA, O..."
1,AirportFrom,152,"[MCO, SAT, LAX, LIT, ATL, MKE, CVG, MOB, CLT, ..."
2,AirportTo,156,"[BDL, BNA, DEN, PHX, BWI, DFW, JAX, MSY, ATL, ..."


Como hay una dimensionalidad muy alta en las categoricas, vamos a utilizar una trasnformacion que no sea one hot , por ende utilizaremos Label encoder

### Generacion de modelos

* Realizamos un split
* Transformacion de valores de preproceso para train dataset
* entrenamiento
* Transformacion en base a lo anterior en el test
* prediccion
* metrica



In [25]:
airlines.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
363020,363021,DL,2582,MCO,BDL,2,1155,153,0
422163,422164,WN,2772,SAT,BNA,6,645,125,0
318247,318248,MQ,3044,LAX,DEN,7,725,140,0
260117,260118,WN,3496,LIT,PHX,4,420,195,0
294548,294549,DL,1924,ATL,BWI,5,1290,113,0


In [23]:
#chequeo de desbalanceo
airlines.Delay.value_counts(normalize=True)

0    0.554217
1    0.445783
Name: Delay, dtype: float64

In [27]:
# split input y target

X = airlines.drop('Delay', axis=1)
y = airlines['Delay']

#split de modelo
from sklearn.model_selection import train_test_split

X_train , X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42, stratify=y)

print(X_train.shape, y_train.shape)

(863, 8) (863,)


Con la trasnformacion vamos a utilizar el LabelEnoder que lo que realiza es poder cambiar cada string por un numero, no es muy recomendable usarlo para categoricas no ordenadas , pero queremos realizarlo para poder ver su uso

In [28]:
from sklearn.preprocessing import OrdinalEncoder#orden natural

#generacion del encoder
le = OrdinalEncoder(handle_unknown='use_encoded_value' , #use cuando tenes valores desconocidos
                    unknown_value=-1) # asignacion a valores desconocidos

# generacion de numeros por cada categorico
le.fit(X_train[cat_columns])

#transformacion de las categoricas en train dataset
X_train[cat_columns] = le.transform(X_train[cat_columns])

In [31]:
#generacion del encoder
le = OrdinalEncoder(handle_unknown='use_encoded_value' , #use cuando tenes valores desconocidos
                    unknown_value=-1) # asignacion a valores desconocidos

# generacion de numeros por cada categorico
le.fit(X_test[cat_columns])

#transformacion de las categoricas en train dataset
X_test[cat_columns] = le.transform(X_test[cat_columns])

In [29]:
X_train.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length
526002,526003,1.0,2306,40.0,38.0,5,450,145
404098,404099,15.0,801,108.0,19.0,5,420,255
360053,360054,15.0,460,109.0,19.0,2,1000,70
307181,307182,10.0,4589,92.0,76.0,6,1015,165
316956,316957,12.0,4839,0.0,132.0,7,660,115


In [30]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 863 entries, 526002 to 205438
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           863 non-null    int64  
 1   Airline      863 non-null    float64
 2   Flight       863 non-null    int64  
 3   AirportFrom  863 non-null    float64
 4   AirportTo    863 non-null    float64
 5   DayOfWeek    863 non-null    int64  
 6   Time         863 non-null    int64  
 7   Length       863 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 60.7 KB


#Entrenamiento del modelo SVM

In [32]:
from sklearn.svm import SVC

#entrenamiento
classifier = SVC(kernel= 'linear')#rbf, polynomial

classifier.fit(X_train, y_train)



SVC(kernel='linear')

In [33]:
pred = classifier.predict(X_test)

testing = pd.DataFrame({'y_true':y_test, 'pred':pred})
#metrica

In [34]:
testing

,y_true,pred
246166,1,0
5740,1,0
379252,1,1
183878,0,0
140527,0,0
...,...,...
349839,1,0
297099,1,0
514783,1,1
115366,1,0


##Metrica

Evaluacion final del modelo en el test



In [35]:
from sklearn.metrics import classification_report, f1_score

print(classification_report(testing['y_true'], testing['pred']))

              precision    recall  f1-score   support

           0       0.61      0.59      0.60       120
           1       0.51      0.53      0.52        96

    accuracy                           0.56       216
   macro avg       0.56      0.56      0.56       216
weighted avg       0.57      0.56      0.57       216



In [36]:
f1_score(testing['y_true'], testing['pred'])

0.5204081632653061